In [ ]:
!pip install PyMultiDictionary spacy googletrans==4.0.0-rc1
!python -m spacy download en_core_web_sm

In [ ]:
from googletrans import Translator
from ipywidgets import widgets
from PyMultiDictionary import MultiDictionary
from PyMultiDictionary import DICT_WORDNET
from PyMultiDictionary import DICT_EDUCALINGO
from IPython.display import clear_output
import requests
from IPython.display import display, Audio, HTML
import ipywidgets as widgets
import inflect

In [ ]:
lang = ['Choose Language','English (en)', 'German (de)', 'French (fr)']
dictionary = MultiDictionary()
translator = Translator()

dropdown = widgets.Dropdown(options=lang, description='Select a Language:')
text_input = widgets.Text(description='Enter a Word:')

# Define variables to store language code and entered word
code = None
word = None

def extract_language_code(option):
    # Extract the two-letter code from the option
    code = option.split('(')[-1].strip(')')
    return code

def on_change(change):
    global code  # Declare code as global to modify its value within the function
    if change['type'] == 'change' and change['name'] == 'value':
        selected_lang = change['new']
        code = extract_language_code(selected_lang)
        print("Selected Language Code:", code)

def on_text_submit(text):
    global word  # Declare word as global to modify its value within the function
    word = text.value
    print("Entered Word:", word)
    pronunciations = get_pronunciations(word, language=code)
    if pronunciations:
        print("Pronunciations for", word)
        for idx, pronunciation in enumerate(pronunciations['items']):
            audio_url = pronunciation['pathmp3']
            try:
                display(HTML(f'<audio controls><source src="{audio_url}" type="audio/mpeg"></audio>'))
            except Exception as e:
                print("Error displaying audio:", e)
    else:
        print("No pronunciations found.")
    if (dictionary.meaning(code, word) == ([], '', '')):
      word = translate_and_identify_singular(word, src_lang=code)

    print(dictionary.meaning(code, word))


def get_pronunciations(word, language='en', sex='m', limit=1, group_in_languages=None):
    url = 'https://apifree.forvo.com/key/09a059a0f3086d3ab04454f0dc199531/format/json/action/word-pronunciations/word/'
    params = {
        'word': word,
        'language': language,
        'sex': sex,
        'limit': limit,
        'group-in-languages': group_in_languages
    }
    response = requests.get(url + word + '/language/' + language + '/sex/' + str(sex) + '/limit/1/')
    if response.status_code == 200:
        return response.json()
    else:
        print("Error occurred:", response.text)
        return None

def find_singular(word, language='en'):
    p = inflect.engine()
    return p.singular_noun(word) or word

def translate_and_identify_singular(word, src_lang='fr', dest_lang='en'):
  if (src_lang == 'en'):
    return find_singular(word)
  translation = translator.translate(word, src=src_lang, dest=dest_lang)
  translated_word = translation.text
  singular_word = find_singular(translated_word)
  translation = translator.translate(singular_word, src=dest_lang, dest=src_lang)
  word = translation.text
  return word

dropdown.observe(on_change)
text_input.on_submit(on_text_submit)

display(dropdown)
display(text_input)


Dropdown(description='Select a Language:', options=('Choose Language', 'English (en)', 'German (de)', 'French …

Text(value='', description='Enter a Word:')

Selected Language Code: de
Entered Word: fra
Error occurred: ["Account disabled."]
No pronunciations found.
(['Noun'], 'Salutation of Italian convent brothers. Anrede italienischer Klosterbrüder.', '')
Entered Word: frau
Error occurred: ["Account disabled."]
No pronunciations found.
(['Pronoun'], 'especially in feminist parlance, otherwise often jokingly for »man«, especially when women are meant. besonders in feministischem Sprachgebrauch, sonst oft scherzhaft für »man«, besonders wenn Frauen gemeint sind.', '')
